In [1]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from tagger.core.mongo.models.order import Order, OrderType, OrderCodeMap, Payment
from tagger.models.inventory import Inventory, InventoryStatus
from tagger.core.label.print_label import print_label
from tagger.core.label.shipping_label import make_shipping_label
from bson import ObjectId

def ready_for_shipping(order_id: str, inventory: Inventory):
    order = Order.objects(id=ObjectId(order_id)).first()
    inventory.out_order_id = order_id
    inventory.status = InventoryStatus.SHIPPING_PENDING
    inventory.save()
    print_label(make_shipping_label(order, inventory))



{('컬러블록스웻셔츠(스모크블루)-SYSTEM',
  '90(F) '): [<Inventory: Inventory #77 [IN_STOCK] 컬러 블록 스웻셔츠(스모크 블루) (211108-012-R)>],
 ('[Re-order]알파카블렌드니트가디건(스모크핑크)-TIME',
  '90(F)'): [<Inventory: Inventory #25 [IN_STOCK] [Re-order] 알파카 블렌드 니트 가디건(스모크 핑크) (201104)>],
 ('[기획]핸드메이드벨티드코트-SJSJ',
  '82(55)'): [<Inventory: Inventory #26 [IN_STOCK] [기획] 핸드메이드 벨티드 코트 (201104)>],
 ('[빈폴스포츠]듀얼후드코어쇼트다운(옐로우)-BEANPOLEL',
  '85'): [<Inventory: Inventory #27 [IN_STOCK] [빈폴스포츠] 듀얼 후드 코어 쇼트 다운(옐로우) (201104)>],
 ('러플칼라블라우스(페일베이지)-MINE',
  '82(55)'): [<Inventory: Inventory #28 [IN_STOCK] 러플 칼라 블라우스(페일 베이지) (201104)>],
 ('리브블록후드구스다운점퍼-MINE',
  '82(55)'): [<Inventory: Inventory #29 [IN_STOCK] 리브 블록 후드 구스다운 점퍼 (201104)>],
 ('볼륨소매니트풀오버(민트)-DEWL',
  'FREE(99)'): [<Inventory: Inventory #30 [IN_STOCK] 볼륨 소매 니트 풀오버(민트) (201104)>],
 ('스티치디테일핸드메이드코트(그린)-CCCOLLECT',
  '88(66)'): [<Inventory: Inventory #31 [IN_STOCK] 스티치 디테일 핸드메이드 코트(그린) (201104)>],
 ('스플릿드레스-TIME',
  '82(55)'): [<Inventory: Inventory #32 [IN_STOCK] 스플릿 드레스 (201104)

In [24]:
in_stock_query = Inventory.objects.filter(status=InventoryStatus.IN_STOCK).order_by("id")
inventories = in_stock_query.all()
orders = (
            Order.objects(
                orderstatus__in=[
                    "PAYMENT_FINISHED",
                    "PRODUCT_PREPARING",
                    "DELIVERY_PREPARING",
                ],
            )
            .order_by("id")
            .all()
        )
inventory_map = {}
for inv in inventories:
    key = (inv.product_name.replace(" ", "") + "-" + inv.product_brand, inv.size.replace(" ", ""))
    if key in inventory_map:
        inventory_map[key].append(inv)
    else:
        inventory_map[key] = [inv]
        
order_item_inventory_matches = []

for order in orders:
    # Skip if already fully fulfilled
    if len(order.orders) == Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING, out_order_id=str(order.id)).count():
        continue
    
    payment = order.payment
    
    
    for item in order.orders:
        p = item.alloffproduct if item.alloffproduct is not None else item.product
        
        direct_matches = [x for x in in_stock_query.filter(
            product_id=p._id
        ).exclude(
            id__in=exhausted_inventory_ids
        ).all() if x.size.replace(" ", "") == item.size.replace(" ", "")]

        if len(direct_matches) > 0:
            match = direct_matches[0]
            exhausted_inventory_ids.append(match.id)
            order_item_inventory_matches.append((order, item, match))
            
        else:
            stringmatch_key = (p.name.replace(" ", "") + "-" + p.brand.keyname, item.size.replace(" ", ""))
            if stringmatch_key in inventory_map:
                matches = inventory_map[stringmatch_key]
                
                match = matches[0]
                exhausted_inventory_ids.append(match.id)
                order_item_inventory_matches.append((order, item, match))
                
                if len(matches) > 0:
                    inventory_map[stringmatch_key] = matches[1:]
                else:
                    del inventory_map[stringmatch_key]
            else:
                # no hope
                pass
            
for order, item, inv in order_item_inventory_matches:
    payment = order.payment
    print("---------------------------------------------------------------------------")
    print(order.id, order.orderstatus, payment.buyername, payment.name, inv.size, order.created, inv.id, inv.code)

---------------------------------------------------------------------------
618a399726685ff051755d16 DELIVERY_PREPARING 윤숙종 오픈 칼라 니트 탑 90(F) 2021-11-09 09:04:23.947000 110 211109-022-R
---------------------------------------------------------------------------
618a3df226685ff051755d37 DELIVERY_PREPARING 이지현 케이프 레이어 풀오버(민트) FREE(99) 2021-11-09 09:22:58.012000 109 211109-021-R
---------------------------------------------------------------------------
618a40c526685ff051755d44 DELIVERY_PREPARING 백경희 러플 칼라 블라우스(페일 베이지) 82(55) 2021-11-09 09:35:01.203000 28 201104-001-R
---------------------------------------------------------------------------
618a667d26685ff051755dad DELIVERY_PREPARING 이유정 캐시미어 니트 롱 드레스  90(F) 2021-11-09 12:15:57.444000 37 201104-003-R
---------------------------------------------------------------------------
618a691b26685ff051755db6 DELIVERY_PREPARING 최희영 울 캐시미어 하이넥 니트 드레스  95 2021-11-09 12:27:07.612000 34 201104-002-R


In [21]:
inventory_map

{('[Re-order]알파카블렌드니트가디건(스모크핑크)-TIME',
  '90(F)'): [<Inventory: Inventory #25 [IN_STOCK] [Re-order] 알파카 블렌드 니트 가디건(스모크 핑크) (201104)>],
 ('[기획]핸드메이드벨티드코트-SJSJ',
  '82(55)'): [<Inventory: Inventory #26 [IN_STOCK] [기획] 핸드메이드 벨티드 코트 (201104)>],
 ('[빈폴스포츠]듀얼후드코어쇼트다운(옐로우)-BEANPOLEL',
  '85'): [<Inventory: Inventory #27 [IN_STOCK] [빈폴스포츠] 듀얼 후드 코어 쇼트 다운(옐로우) (201104)>],
 ('러플칼라블라우스(페일베이지)-MINE', '82(55)'): [],
 ('리브블록후드구스다운점퍼-MINE',
  '82(55)'): [<Inventory: Inventory #29 [IN_STOCK] 리브 블록 후드 구스다운 점퍼 (201104)>],
 ('볼륨소매니트풀오버(민트)-DEWL',
  'FREE(99)'): [<Inventory: Inventory #30 [IN_STOCK] 볼륨 소매 니트 풀오버(민트) (201104)>],
 ('스티치디테일핸드메이드코트(그린)-CCCOLLECT',
  '88(66)'): [<Inventory: Inventory #31 [IN_STOCK] 스티치 디테일 핸드메이드 코트(그린) (201104)>],
 ('스플릿드레스-TIME',
  '82(55)'): [<Inventory: Inventory #32 [IN_STOCK] 스플릿 드레스 (201104)>],
 ('아노락퀼팅점퍼(오렌지)-THECASHMERE',
  '82(F)'): [<Inventory: Inventory #33 [IN_STOCK] 아노락 퀼팅 점퍼(오렌지) (201104)>],
 ('울캐시미어하이넥니트드레스-BEANPOLEL', '95'): [],
 ('워싱코튼패딩점퍼(그레이)-CCCOLLECT',
  '

In [7]:

    



#         for iid in inv_ids:
#             ready_for_shipping(order.id,
#                    Inventory.objects.get(id=iid)
#                   )
    

더블M머플러-MAJE
울캐시미어블렌드니트-VINCE
[택제거상품]Teca-MAXMARA
[WeekendMaxMara]PATNER(Camel)-MAXMARA
[MEN](7colors)울캐시미어스웨터-POLO
컬러블록페이크무스탕자켓(아이보리)-CCCOLLECT
[로렌랄프로렌]서플리스데님미디드레스-POLO
네이비단추FIRE니트-SANDRO
네이비CLUB맨투맨-SANDRO
[MEN](7colors)울캐시미어스웨터-POLO
워싱코튼패딩점퍼(그레이)-CCCOLLECT
[택제거상품]Teca-MAXMARA
Harrowden(올리브)-BURBERRY
Ashbycanvasbucketbag(블랙/미듐)-BURBERRY
Harrowden(밍크그레이)-BURBERRY
Tortosweatshirt(그레이)-BURBERRY
Frankbyquiltedjacket(블랙)-BURBERRY
Frankbyquiltedjacket(블랙)-BURBERRY
Ashbycanvasbucketbag(블랙/미듐)-BURBERRY
[MEN](7colors)울캐시미어스웨터-POLO
Harrowden(올리브)-BURBERRY
벨티드니트드레스-TIME
[재입고]Ashbycanvasbucketbag(블랙/미듐)-BURBERRY
[로렌랄프로렌]프린티드터틀넥탑-POLO
캐시미어더블코트-SYSTEM
Tortosweatshirt(네이비)-BURBERRY
[MEN](7colors)울캐시미어스웨터-POLO
[로렌랄프로렌]브로드클로스블루종슬리브셔츠-POLO
[재입고]래시프린지캐시미어울머플러-BURBERRY
[MEN](7colors)울캐시미어스웨터-POLO
언밸런스울스커트-SYSTEM
컨버터블셔츠-VINCE
벨티드후드코트(멜란지베이지)-CCCOLLECT
VISIDOR트위드자켓-MAJE
핸드메이드캐시미어코트-BEANPOLEL
컨버터블셔츠-VINCE
컨버터블셔츠-VINCE
핸드메이드캐시미어코트-BEANPOLEL
[로렌랄프로렌]브로드클로스블루종슬리브셔츠-POLO
벨티드플랩코트(카멜)-CCCOLLECT
숄블록니트탑-TIME
[택제거상품]

In [11]:
# ready_for_shipping("618516ef26685ff0517554c5",
#                    Inventory.objects.get(id=92)
#                   )

In [19]:
ready_for_shipping("6185018c26685ff051755440",
                   Inventory.objects.get(code="211109-010")
                  )

In [7]:
ready_for_shipping(str(Order.get_by_code("bq3nu").id),
                   Inventory.objects.get(code="211109-007")
                  )

In [2]:
to_be_shipped = Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING).all()

In [7]:
df = pd.DataFrame([{
    "order_id": x.out_order_id,
    "name": x.out_order.get_payment().buyername,
    "order_name": x.out_order.get_payment().name,
    "inventory_code": x.code
} for x in to_be_shipped])

In [8]:
df.to_clipboard()

In [9]:
Inventory.objects.filter(out_order_id="6188765726685ff051755a57").first().id

70

In [8]:
inv_keys

{77: '컬러블록스웻셔츠(스모크블루)-SYSTEM',
 25: '[Re-order]알파카블렌드니트가디건(스모크핑크)-TIME',
 26: '[기획]핸드메이드벨티드코트-SJSJ',
 27: '[빈폴스포츠]듀얼후드코어쇼트다운(옐로우)-BEANPOLEL',
 28: '러플칼라블라우스(페일베이지)-MINE',
 29: '리브블록후드구스다운점퍼-MINE',
 30: '볼륨소매니트풀오버(민트)-DEWL',
 31: '스티치디테일핸드메이드코트(그린)-CCCOLLECT',
 32: '스플릿드레스-TIME',
 33: '아노락퀼팅점퍼(오렌지)-THECASHMERE',
 34: '울캐시미어하이넥니트드레스-BEANPOLEL',
 35: '워싱코튼패딩점퍼(그레이)-CCCOLLECT',
 36: '이너드레스-TIME',
 37: '캐시미어니트롱드레스-SYSTEM',
 38: '캐시미어니트롱드레스-SYSTEM',
 39: '캐시미어니트롱드레스-SYSTEM',
 40: '캐시미어니트롱드레스-SYSTEM',
 41: '캐시미어니트롱드레스-SYSTEM',
 42: '컬러블록페이크무스탕자켓(그레이)-CCCOLLECT',
 43: '프레이드트위드드레스(다크네이비)-CCCOLLECT',
 44: '핸드메이드더블코트-SYSTEM',
 45: '핸드메이드더블코트-SYSTEM',
 46: '핸드메이드더블코트-TIME',
 47: '핸드메이드더블코트-TIME',
 52: '벨티드후드구스다운점퍼-SYSTEM',
 53: '[Re-order]부클레트위드드레스-MINE',
 58: '★럭키템★핸드메이드벨티드더블코트C204MSG584-CCCOLLECT',
 59: '스티치디테일핸드메이드코트(그린)-CCCOLLECT',
 60: '퀄팅구스다운점퍼-SYSTEM',
 73: '언밸런스후드점퍼-SYSTEM',
 74: '벨티드플랩코트(카멜)-CCCOLLECT',
 75: '테일러드더블코트(블랙)-DEWL'}